In [2]:
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores.faiss import FAISS

embeddings = OpenAIEmbeddings()
vectorstore = FAISS.load_local("smartlearn-index",embeddings,allow_dangerous_deserialization=True)



In [18]:

from typing import Optional
from langchain.tools import BaseTool
from langchain.callbacks.manager import AsyncCallbackManagerForToolRun, CallbackManagerForToolRun

class CustomSearchTool(BaseTool):
    name = "smartlearnsupport"
    description = "useful for when you need to answer questions about smartlearnsupport"

    def _run(self, query: str, run_manager: Optional[CallbackManagerForToolRun] = None) -> str:
        store = vectorstore.as_retriever()
        docs = store.get_relevant_documents(query)
        text_list = [doc.page_content for doc in docs]
        #print("############# ",text_list)
        return "\n".join(text_list)

    async def _arun(self, query: str, run_manager: Optional[AsyncCallbackManagerForToolRun] = None) -> str:
        """Use the tool asynchronously."""
        raise NotImplementedError("custom_search does not support async")

In [24]:
from langchain.agents import AgentType,create_openai_tools_agent,AgentExecutor
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate,MessagesPlaceholder
from langchain import hub
llm=ChatOpenAI()
tools = [CustomSearchTool()]
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", """
          You are an agent for smartlearn platform that responds based on the input 
        example:
          Human: Whats is agent aboutt?
          AI: I respond course related queries
          """),
        MessagesPlaceholder("chat_history", optional=True),
        ("human", "{input}"),
        MessagesPlaceholder("agent_scratchpad"),
    ]
)
agent = create_openai_tools_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)


In [25]:
print(agent_executor.invoke({"input": "Hi"}))
print(agent_executor.invoke({"input": "What courses are offered by you"}))
print(agent_executor.invoke({"input": "What is the upcoming schedule"}))
print(agent_executor.invoke({"input": "Do you offer AI related training"}))
print(agent_executor.invoke({"input": "I want information about refund"}))



> Entering new AgentExecutor chain...
Hello! How can I assist you today?

> Finished chain.
{'input': 'Hi', 'output': 'Hello! How can I assist you today?'}


> Entering new AgentExecutor chain...

Invoking: `smartlearnsupport` with `list of courses`


#############  ['in the course prerequisites.', 'Q: What types of courses does SmartLearn offer?', 'A: Some courses might require specific tools or software, which are detailed in the course', 'A: Courses are designed by experts and routinely reviewed for quality.']
in the course prerequisites.
Q: What types of courses does SmartLearn offer?
A: Some courses might require specific tools or software, which are detailed in the course
A: Courses are designed by experts and routinely reviewed for quality.SmartLearn offers a variety of courses designed by experts and routinely reviewed for quality. Some courses may have specific prerequisites or require certain tools or software, which are detailed in the course descriptions. If you are inter